# Training a Neural Network
We'll be training a neural network using particle swarm optimization. For this we'll be using the standard global-best PSO `pyswarms.single.GBestPSO` for optimizing the network's weights and biases.

We will show the performance of training LeNet-5 using only PSO as its optimizer. As such, no gradient information is used.

We will show the performance using an hour of runtime on both Fashion-MNIST and CIFAR-10, two simple benchmark datasets.

In [ ]:
### SETTINGS

## Dataset. For convenience, comment out the one you do not wish to use.
# --------------
dataset_to_use = "FashionMNIST"
# dataset_to_use = "CIFAR"


## Runtime. Time to run PSO for, in minutes.
# (Note: the current iteration will be completed, so technically it will always run a little longer)
# --------------
runtime = 1


## PSO settings.
# --------------
c1 = 1.5  # personal best /cognitive parameter
c2 = 1.5  # global best   /social parameter
w = 0.93  # inertia

num_particles = 5
num_iterations = 100000   # Very large because the runtime setting should terminate PSO well before the number here is reached.

In [ ]:
# Import modules
import numpy as np
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
from torch.utils.data import DataLoader

import torch.nn.functional as F
from torchvision import datasets, transforms


# Import PySwarms
!pip install pyswarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## By default, PySwarms did not allow for time limits during PSO runs. Therefore, we modified the optimize function of the GlobalBestPSO class in this library. As a result of this modification, we can provide the maximum runtime in minutes (time).

In [ ]:
# Manual overwrite of PySwarms to print timestamps.

import logging
import datetime
from collections import deque
from pyswarms.base import SwarmOptimizer
from pyswarms.utils import Reporter
from pyswarms.backend.topology import Star
from pyswarms.backend.operators import compute_pbest, compute_objective_function 
from pyswarms.backend.handlers import BoundaryHandler, VelocityHandler, OptionsHandler

class GlobalBestPSO(SwarmOptimizer):
    def __init__(
        self,
        n_particles,
        dimensions,
        options,
        bounds=None,
        oh_strategy=None,
        bh_strategy="periodic",
        velocity_clamp=None,
        vh_strategy="unmodified",
        center=1.00,
        ftol=-np.inf,
        ftol_iter=1,
        init_pos=None,
    ):
        """Initialize the swarm. Modified for this particular project.
        """
        super(GlobalBestPSO, self).__init__(
            n_particles=n_particles,
            dimensions=dimensions,
            options=options,
            bounds=bounds,
            velocity_clamp=velocity_clamp,
            center=center,
            ftol=ftol,
            ftol_iter=ftol_iter,
            init_pos=init_pos,
        )

        if oh_strategy is None:
            oh_strategy = {}
        # Initialize logger
        self.rep = Reporter(logger=logging.getLogger(__name__))
        # Initialize the resettable attributes
        self.reset()
        # Initialize the topology
        self.top = Star()
        self.bh = BoundaryHandler(strategy=bh_strategy)
        self.vh = VelocityHandler(strategy=vh_strategy)
        self.oh = OptionsHandler(strategy=oh_strategy)
        self.name = __name__

    def optimize(
        self, objective_func, iters, n_processes=None, verbose=True, time=None, **kwargs
    ):
        """Optimize the swarm for a number of iterations, modified for this particular project.
        """

        # Apply verbosity
        if verbose:
            log_level = logging.INFO
        else:
            log_level = logging.NOTSET

        self.rep.log("Obj. func. args: {}".format(kwargs), lvl=logging.DEBUG)
        self.rep.log(
            "Optimize for {} iters with {}".format(iters, self.options),
            lvl=log_level,
        )
        # Populate memory of the handlers
        self.bh.memory = self.swarm.position
        self.vh.memory = self.swarm.position

        # Setup Pool of processes for parallel evaluation
        pool = None if n_processes is None else mp.Pool(n_processes)

        self.swarm.pbest_cost = np.full(self.swarm_size[0], np.inf)
        ftol_history = deque(maxlen=self.ftol_iter)

        best_positions = []
        # 
        if verbose == False and time != None:
            # If provided, set the max runtime
            endTime = datetime.datetime.now() + datetime.timedelta(minutes=time)

            i = 0
            while datetime.datetime.now() <= endTime:
                if i%10==0:
                  secondsLeft = (endTime - datetime.datetime.now()).seconds
                  print(secondsLeft, "seconds left..")
                  print("Iteration: ",i)
                self.swarm.current_cost = compute_objective_function(self.swarm, objective_func, pool=pool, **kwargs)
                self.swarm.pbest_pos, self.swarm.pbest_cost = compute_pbest(self.swarm)
                # Set best_cost_yet_found for ftol
                best_cost_yet_found = self.swarm.best_cost
                self.swarm.best_pos, self.swarm.best_cost = self.top.compute_gbest(self.swarm)
                best_positions.append(self.swarm.best_pos)
                print("Best cost: ",round(best_cost_yet_found,3))
                # Save to history
                hist = self.ToHistory(
                    best_cost=self.swarm.best_cost,
                    mean_pbest_cost=np.mean(self.swarm.pbest_cost),
                    mean_neighbor_cost=self.swarm.best_cost,
                    position=self.swarm.position,
                    velocity=self.swarm.velocity,
                )
                self._populate_history(hist)
                # Verify stop criteria based on the relative acceptable cost ftol
                relative_measure = self.ftol * (1 + np.abs(best_cost_yet_found))
                delta = (
                    np.abs(self.swarm.best_cost - best_cost_yet_found)
                    < relative_measure
                )
                if i < self.ftol_iter:
                    ftol_history.append(delta)
                else:
                    ftol_history.append(delta)
                    if all(ftol_history):
                        break
                # Perform options update
                self.swarm.options = self.oh(
                    self.options, iternow=i, itermax=iters
                )
                # Perform velocity and position updates
                self.swarm.velocity = self.top.compute_velocity(
                    self.swarm, self.velocity_clamp, self.vh, self.bounds
                )
                self.swarm.position = self.top.compute_position(
                    self.swarm, self.bounds, self.bh
                )
                i = i + 1
        else:
            print("Removed for this project")
        # Obtain the final best_cost and the final best_position
        final_best_cost = self.swarm.best_cost.copy()
        final_best_pos = self.swarm.pbest_pos[
            self.swarm.pbest_cost.argmin()
        ].copy()
        # Write report in log and return final cost and position
        self.rep.log(
            "Optimization finished | best cost: {}, best pos: {}".format(
                final_best_cost, final_best_pos
            ),
            lvl=log_level,
        )
        # Close Pool of Processes
        if n_processes is not None:
            pool.close()
        return (final_best_cost, final_best_pos, best_positions)

## Load the dataset

In [ ]:
from torchvision import datasets, transforms # This import appears to be bugged, hence why we reload it in multiple cells

def dataset(variant):
  """Loads in appropriate dataset given string.
    Valid options are 'FashionMNIST' and 'CIFAR'.

    Currently restricted to 10000 images for train and validation/test set due to runtime considerations. 
    """
  transformer = transforms.Compose([transforms.ToTensor()])

  if(variant is 'FashionMNIST'):
    # download and create datasets
    train_dataset = datasets.FashionMNIST(root='mnist_data', 
                                  train=True, 
                                  transform=transformer,
                                  download=True)

    valid_dataset = datasets.FashionMNIST(root='mnist_data', 
                                  train=False, 
                                  transform=transformer)
  
  elif(variant is 'CIFAR'):
    train_dataset = datasets.CIFAR10(root='data', 
                               train=True, 
                               transform=transformer,
                               download=True)

    valid_dataset = datasets.CIFAR10(root='data',
                               train=False, 
                               transform=transformer)
  else:
    print("Not a valid dataset")

  train_loader = DataLoader(train_dataset, batch_size=len(train_dataset))
  valid_loader = DataLoader(valid_dataset, batch_size=len(valid_dataset))
  
  X_train = next(iter(train_loader))[0][:10000] #images
  y_train = next(iter(train_loader))[1][:10000] #labels
  X_valid = next(iter(valid_loader))[0][:10000] #images
  y_valid = next(iter(valid_loader))[1][:10000] #labels

  return X_train, y_train, X_valid, y_valid

In [ ]:
X_train, y_train, X_valid, y_valid = dataset(dataset_to_use)

In [ ]:
print(X_train.shape)

torch.Size([10000, 1, 28, 28])


# Create the Network


In [ ]:
f_dim = 5   # filters are always 5x5

# Number of filters (conv) or neurons (dense)
conv1 = 6
conv2 = 16
dense1 = 120
dense2 = 84

n_classes = 10

num_samples = X_train.shape[0]

In [ ]:
# in_channels = number of channels in input data
# dimensions = total number of weights + biases in the neural network

in_channels = 0
dimensions = 0
if dataset_to_use is "FashionMNIST":
    dimensions = 61706
    in_channels = 1
elif dataset_to_use is "CIFAR":
    dimensions = 548878
    in_channels = 3
else:
    in_channels = 1

In [ ]:
# Number of weights (Wx) per layer (biases separate (bx)):
len_W1 = f_dim * f_dim * conv1              * in_channels * in_channels
len_b1 = conv1                              * in_channels
len_W2 = f_dim * f_dim * conv2 * conv1      * in_channels * in_channels
len_b2 = conv2                              * in_channels
len_W3 = f_dim * f_dim * conv2 * dense1     * in_channels * in_channels
len_b3 = dense1                             * in_channels
len_W4 = dense1 * dense2                    * in_channels * in_channels
len_b4 = dense2                             * in_channels
len_W5 = dense2 * n_classes                 * in_channels
len_b5 = n_classes

total_post_1 = len_W1 + len_b1
total_post_2 = total_post_1 + len_W2 + len_b2
total_post_3 = total_post_2 + len_W3 + len_b3
total_post_4 = total_post_3 + len_W4 + len_b4
total_post_5 = total_post_4 + len_W5 + len_b5

In [ ]:
def logits_function(p, images):
  """Given an array of weights of the appropriate length, creates LeNet and performs a forward pass on the train data."""
  class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Reshape weight input to network size
        W1 = p[0:len_W1].reshape((conv1*in_channels, 1 * in_channels, 5, 5))
        W1 = torch.Tensor(W1)
        b1 = p[len_W1:total_post_1].reshape((conv1*in_channels,))
        b1 = torch.Tensor(b1)

        W2 = p[total_post_1:(total_post_1+len_W2)].reshape((conv2*in_channels, conv1 * in_channels, 5, 5))
        b2 = p[(total_post_1+len_W2):total_post_2].reshape((conv2 *in_channels,))
        W2 = torch.Tensor(W2)
        b2 = torch.Tensor(b2)

        W3 = p[total_post_2:(total_post_2+len_W3)].reshape((dense1*in_channels, conv2 * in_channels, 5, 5))
        b3 = p[(total_post_2+len_W3):total_post_3].reshape((dense1 *in_channels,))
        W3 = torch.Tensor(W3)
        b3 = torch.Tensor(b3)

        W4 = p[total_post_3:(total_post_3+len_W4)].reshape((dense2*in_channels, dense1 * in_channels)) 
        b4 = p[(total_post_3+len_W4):total_post_4].reshape((dense2 *in_channels,))
        W4 = torch.Tensor(W4)
        b4 = torch.Tensor(b4)

        W5 = p[total_post_4:(total_post_4+len_W5)].reshape((n_classes, dense2 * in_channels))
        b5 = p[(total_post_4+len_W5):total_post_5].reshape((n_classes,))
        W5 = torch.Tensor(W5)
        b5 = torch.Tensor(b5)

        # Define activation and pool operations
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))

        # Define architecture and map weights
        if dataset_to_use is "FashionMNIST":
          self.conv1 = nn.Conv2d(in_channels, out_channels=conv1 * in_channels, kernel_size=(5,5),stride=(1,1),padding=(2,2))
        elif dataset_to_use is "CIFAR":
          self.conv1 = nn.Conv2d(in_channels, out_channels=conv1 * in_channels, kernel_size=(5,5),stride=(1,1),padding=(0,0))
        else:
          print("Invalid dataset, cannot determine padding.")
        self.conv1.weight.data = W1
        self.conv1.bias.data = b1

        self.conv2 = nn.Conv2d(conv1 * in_channels, out_channels=conv2 * in_channels, kernel_size=(5,5),stride=(1,1),padding=(0,0))
        self.conv2.weight.data = W2
        self.conv2.bias.data = b2

        self.conv3 = nn.Conv2d(conv2 * in_channels, out_channels=dense1 * in_channels, kernel_size=(5,5),stride=(1,1),padding=(0,0))
        self.conv3.weight.data = W3
        self.conv3.bias.data = b3

        self.linear1 = nn.Linear(dense1 * in_channels, dense2 * in_channels)
        self.linear1.weight.data = W4
        self.linear1.bias.data = b4

        self.linear2 = nn.Linear(dense2 * in_channels, n_classes)
        self.linear2.weight.data = W5
        self.linear2.bias.data = b5
    
    def forward(self, x):
      # Perform a forward pass of the network
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x)) # num_examples x 120 x 1 x 1 --> num_examples x 120
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

  model = LeNet()
  return model(images)

In [ ]:
# Forward propagation
def forward_prop(params):
    """Computes for the forward propagation of the neural network from an array of weights and biases.
    Returns the loss.
    """

    logits = logits_function(params, X_train)

    # Compute for the softmax of the logits
    probs = F.softmax(logits, dim=1)
    
    # Compute the loss from the (softmaxed) logit probs
    loss = nn.CrossEntropyLoss()
    y_train_tensor = torch.tensor(y_train)
    y_train_tensor = y_train_tensor.type(torch.LongTensor)
    output = loss(probs, y_train_tensor)
    
    return output.item()

In [ ]:
def f(x):
    """Forward propagation for the 
    whole swarm. Method adapted from the framework.
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)
    

# Run the Network

In [ ]:
%%time
# Initialize swarm
options = {'c1': c1, 'c2': c2, 'w': w}

# Call instance of PSO
optimizer = GlobalBestPSO(n_particles=num_particles, dimensions=dimensions, options=options)

# Perform optimization
# Verbose must be false for time to work
cost, pos, best_pos = optimizer.optimize(f, iters=num_iterations, verbose=False, time=runtime)

59 seconds left..
Iteration:  0


TypeError: ignored

# Report the Performance

In [ ]:
def predict(pos, images):
    """
    Use the trained weights to perform class predictions.
    """
    
    logits = logits_function(pos, images)
    logits = logits.detach().numpy()
    y_pred = np.argmax(logits, axis=1)

    return y_pred

In [ ]:
print("Best train acc: ",max([(predict(x, X_train) == y_train.numpy()).mean() for x in best_pos]))

In [ ]:
print("Best val acc: ", max([(predict(x, X_valid) == y_valid.numpy()).mean() for x in best_pos]))